In [1]:
%load_ext sql
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

In [2]:
%sql sqlite:///library.db

Connecting to 'sqlite:///library.db'

In [4]:
%%sql
CREATE TABLE Library (
    library_name TEXT,
    address TEXT, 
    phone_number TEXT,
    email TEXT,
    PRIMARY KEY (library_name, address)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [5]:
%%sql
CREATE TABLE Item (
    item_id INTEGER PRIMARY KEY,
    category_id INTEGER,
    library_name TEXT,
    address TEXT,
    title TEXT,
    status TEXT,
    genre TEXT,
    location TEXT,
    publisher_name TEXT,
    FOREIGN KEY (category_id)
        REFERENCES ItemCategory (category_id)
    FOREIGN KEY (library_name)
        REFERENCES Library (library_name),
    FOREIGN KEY (address) 
        REFERENCES Library (address)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [6]:
%%sql
CREATE TABLE ItemCategory (
    category_id INTEGER PRIMARY KEY,
    category_name TEXT
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [7]:
%%sql
CREATE TABLE Reading (
    item_id INTEGER PRIMARY KEY,
    isbn INTEGER,
    author TEXT,
    FOREIGN KEY (item_id)
        REFERENCES Item (item_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [8]:
%%sql
CREATE TABLE Music (
    item_id INTEGER PRIMARY KEY,
    artist TEXT,
    num_songs INTEGER,
    FOREIGN KEY (item_id)
        REFERENCES Item (item_id)
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [9]:
%%sql
CREATE TABLE User (
    user_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    birthdate DATE NOT NULL,
    phone_number TEXT NOT NULL,
    address TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE,
    member_since DATE DEFAULT (datetime('now', 'localtime')),
    favourite_genre TEXT
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [10]:
%%sql
CREATE TABLE Employee (
    user_id INTEGER PRIMARY KEY,
    job_title TEXT,
    salary INTEGER,
    FOREIGN KEY (user_id)
        REFERENCES User (user_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [11]:
%%sql
CREATE TABLE Events (
    event_id INTEGER PRIMARY KEY,
    library_name TEXT,
    address TEXT,
    room_number INTEGER,
    description TEXT,
    event_date DATE,
    start_time TEXT,
    end_time TEXT,
    FOREIGN KEY (library_name, address)
        REFERENCES Library (library_name, address)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [50]:
%%sql
CREATE TABLE Audience (
    audience_id INTEGER PRIMARY KEY,
    type TEXT,
    genre TEXT
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [15]:
%%sql
CREATE TABLE Borrows (
    user_id INTEGER,
    item_id INTEGER,
    borrow_date DATE DEFAULT (datetime('now', 'localtime')),
    due_date DATE DEFAULT (datetime('now', '+14 days', 'localtime')),
    return_date DATE,
    fine INTEGER DEFAULT 20,
    PRIMARY KEY (user_id, item_id, borrow_date),
    FOREIGN KEY (user_id)
        REFERENCES User (user_id),
    FOREIGN KEY (item_id)
        REFERENCES Item (item_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [16]:
%%sql
CREATE TABLE Attending (
    user_id INTEGER,
    event_id INTEGER,
    PRIMARY KEY (user_id, event_id),
    FOREIGN KEY (user_id)
        REFERENCES User (user_id),
    FOREIGN KEY (event_id)
        REFERENCES Events (event_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [17]:
%%sql
CREATE TABLE RecommendedFor (
    event_id INTEGER,
    audience_id INTEGER,
    PRIMARY KEY (event_id, audience_id),
    FOREIGN KEY (event_id)
        REFERENCES Events (event_id),
    FOREIGN KEY (audience_id)
        REFERENCES Audience (audience_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [18]:
%%sql
CREATE TABLE BelongsTo (
    user_id INTEGER,
    audience_id INTEGER,
    PRIMARY KEY (user_id, audience_id),
    FOREIGN KEY (user_id)
        REFERENCES User (user_id),
    FOREIGN KEY (audience_id)
        REFERENCES Audience (audience_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [19]:
%%sql
CREATE TRIGGER UpdateFavGenre AFTER INSERT ON Borrows
FOR EACH ROW
BEGIN
    UPDATE User
    SET favourite_genre = (
        SELECT genre
        FROM Item
        JOIN Borrows ON Item.item_id = Borrows.item_id
        WHERE Borrows.user_id = NEW.user_id
        GROUP BY Item.genre
        ORDER BY COUNT(*) DESC, MAX(Borrows.borrow_date) DESC
        LIMIT 1
    ) WHERE user_id = NEW.user_id;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

In [20]:
%%sql
/*
    Okay to explain this so I dont seem insane:
    1. Upon update of favourite_genre, we need to first calculate the users age to ensure we have their most updated age range
    2. Then, we need to use this Age to determine which "type" of audience they fall into (child, teen, adult). 
    3. Then, we need to check if the combination of "type" and "genre" exists within the entity Audience.
    4. If it does not ecist, we need to insert it into Audience - this allows dynamic adding of Audiences.
    5. Then we need to grab either the newly created, or existing audience's id, and update the BelongTo table with the user_id and audience_id.
*/
CREATE TRIGGER UpdateUserAudience
AFTER UPDATE OF favourite_genre ON User
FOR EACH ROW
WHEN NEW.favourite_genre IS NOT NULL
BEGIN
    DELETE FROM BelongsTo
    WHERE user_id = NEW.user_id;

    INSERT OR IGNORE INTO Audience (type, genre)
    SELECT
        CASE
            WHEN CAST((julianday('now') - julianday(birthdate)) / 365.2422 AS INTEGER) < 13 THEN 'Children'
            WHEN CAST((julianday('now') - julianday(birthdate)) / 365.2422 AS INTEGER) BETWEEN 13 AND 19 THEN 'Teens'
            ELSE 'Adults'
        END AS audience_type,
        NEW.favourite_genre
    FROM User
    WHERE user_id = NEW.user_id
    AND NOT EXISTS (
        SELECT 1
        FROM Audience
        WHERE
            type = CASE
                WHEN CAST((julianday('now') - julianday(birthdate)) / 365.2422 AS INTEGER) < 13 THEN 'Children'
                WHEN CAST((julianday('now') - julianday(birthdate)) / 365.2422 AS INTEGER) BETWEEN 13 AND 19 THEN 'Teens'
                ELSE 'Adults'
            END
            AND genre = NEW.favourite_genre
    );

    -- Update the BelongsTo table with the user_id and audience_id
    INSERT OR REPLACE INTO BelongsTo (user_id, audience_id)
    SELECT
        NEW.user_id,
        Audience.audience_id
    FROM Audience
    WHERE
        type = (
            SELECT CASE
                WHEN CAST((julianday('now') - julianday(birthdate)) / 365.2422 AS INTEGER) < 13 THEN 'Children'
                WHEN CAST((julianday('now') - julianday(birthdate)) / 365.2422 AS INTEGER) BETWEEN 13 AND 19 THEN 'Teens'
                ELSE 'Adults'
            END
            FROM User
            WHERE user_id = NEW.user_id
        )
        AND genre = NEW.favourite_genre;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

# Sample data being generated

In [23]:
%%sql 
INSERT INTO Library (library_name, address, phone_number, email) VALUES
('Burnaby Public Library', '6100 Willingdon Ave', '604-294-7755', 'info@burnabypubliclibrary.ca')

Running query in 'sqlite:///library.db'

1 rows affected.

++
||
++
++

In [24]:
%%sql
INSERT INTO ItemCategory (category_id, category_name) VALUES
(1, 'Book'),
(2, 'Online Book'),
(3, 'Journal'),
(4, 'CD'),
(5, 'Vinyl');

Running query in 'sqlite:///library.db'

5 rows affected.

++
||
++
++

In [25]:
%%sql
INSERT INTO User (user_id, first_name, last_name, birthdate, phone_number, address, email) VALUES
(1, 'John', 'Smith', '1985-07-15', '555-123-4567', '123 Main St', 'john.smith@email.com'),
(2, 'Emily', 'Johnson', '1992-03-22', '555-234-5678', '456 Oak Ave', 'emily.j@email.com'),
(3, 'Michael', 'Williams', '1978-11-30', '555-345-6789', '789 Pine Rd', 'michael.w@email.com'),
(4, 'Sarah', 'Brown', '1995-05-18', '555-456-7890', '321 Elm St', 'sarah.b@email.com'),
(5, 'David', 'Jones', '1982-09-25', '555-567-8901', '654 Maple Dr', 'david.j@email.com');

Running query in 'sqlite:///library.db'

5 rows affected.

++
||
++
++

In [32]:
%%sql
INSERT INTO Item (item_id, category_id, library_name, address, title, status, genre, location, publisher_name) VALUES
-- Books (1-10)
(1, 1, 'Burnaby Public Library', '6100 Willingdon Ave', 'The Great Gatsby', 'Available', 'Fiction', 'Fiction Aisle 1', 'Scribner'),
(2, 1, 'Burnaby Public Library', '6100 Willingdon Ave', 'Pride and Prejudice', 'Available', 'Romance', 'Romance Aisle 1', 'T. Egerton'),
(3, 1, 'Burnaby Public Library', '6100 Willingdon Ave', 'The Hobbit', 'Available', 'Fiction', 'Fiction Aisle 2', 'Allen & Unwin'),
(4, 1, 'Burnaby Public Library', '6100 Willingdon Ave', 'Gone Girl', 'Available', 'Mystery', 'Mystery Aisle 1', 'Crown Publishing'),
(5, 1, 'Burnaby Public Library', '6100 Willingdon Ave', 'The Da Vinci Code', 'Available', 'Mystery', 'Mystery Aisle 2', 'Doubleday'),
(6, 1, 'Burnaby Public Library', '6100 Willingdon Ave', 'The Notebook', 'Available', 'Romance', 'Romance Aisle 2', 'Warner Books'),
(7, 1, 'Burnaby Public Library', '6100 Willingdon Ave', '1984', 'Available', 'Fiction', 'Fiction Aisle 3', 'Secker & Warburg'),
(8, 1, 'Burnaby Public Library', '6100 Willingdon Ave', 'The Girl with the Dragon Tattoo', 'Available', 'Mystery', 'Mystery Aisle 3', 'Norstedts'),
(9, 1, 'Burnaby Public Library', '6100 Willingdon Ave', 'Jane Eyre', 'Available', 'Romance', 'Romance Aisle 3', 'Smith, Elder & Co.'),
(10, 1, 'Burnaby Public Library', '6100 Willingdon Ave', 'To Kill a Mockingbird', 'Available', 'Fiction', 'Fiction Aisle 4', 'J. B. Lippincott'),
-- Music (11-20)
(11, 4, 'Burnaby Public Library', '6100 Willingdon Ave', 'Thriller', 'Available', 'Pop', 'Media Section A1', 'Epic Records'),
(12, 4, 'Burnaby Public Library', '6100 Willingdon Ave', 'Back in Black', 'Available', 'Rock', 'Media Section A2', 'Albert Productions'),
(13, 5, 'Burnaby Public Library', '6100 Willingdon Ave', 'The Dark Side of the Moon', 'Available', 'Rock', 'Media Section B1', 'Harvest'),
(14, 4, 'Burnaby Public Library', '6100 Willingdon Ave', 'Rumours', 'Available', 'Rock', 'Media Section A3', 'Warner Bros.'),
(15, 4, 'Burnaby Public Library', '6100 Willingdon Ave', 'The Marshall Mathers LP', 'Available', 'Hip Hop', 'Media Section C1', 'Aftermath'),
(16, 5, 'Burnaby Public Library', '6100 Willingdon Ave', 'Abbey Road', 'Available', 'Rock', 'Media Section B2', 'Apple'),
(17, 4, 'Burnaby Public Library', '6100 Willingdon Ave', 'The Chronic', 'Available', 'Hip Hop', 'Media Section C2', 'Death Row'),
(18, 4, 'Burnaby Public Library', '6100 Willingdon Ave', 'Born to Run', 'Available', 'Rock', 'Media Section A4', 'Columbia'),
(19, 5, 'Burnaby Public Library', '6100 Willingdon Ave', 'Kind of Blue', 'Available', 'Jazz', 'Media Section B3', 'Columbia'),
(20, 4, 'Burnaby Public Library', '6100 Willingdon Ave', 'The Eminem Show', 'Available', 'Hip Hop', 'Media Section C3', 'Aftermath');

Running query in 'sqlite:///library.db'

20 rows affected.

++
||
++
++

In [35]:
%%sql
INSERT INTO Reading (item_id, isbn, author) VALUES
(1, 9780743273565, 'F. Scott Fitzgerald'),
(2, 9781503290563, 'Jane Austen'),
(3, 9780547928227, 'J.R.R. Tolkien'),
(4, 9780307588371, 'Gillian Flynn'),
(5, 9780307474278, 'Dan Brown'),
(6, 9780446605236, 'Nicholas Sparks'),
(7, 9780451524935, 'George Orwell'),
(8, 9780307269751, 'Stieg Larsson'),
(9, 9780141441146, 'Charlotte Brontë'),
(10, 9780061120084, 'Harper Lee');


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [36]:
%%sql
INSERT INTO Music (item_id, artist, num_songs) VALUES
(11, 'Michael Jackson', 9),
(12, 'AC/DC', 10),
(13, 'Pink Floyd', 10),
(14, 'Fleetwood Mac', 11),
(15, 'Eminem', 18),
(16, 'The Beatles', 17),
(17, 'Dr. Dre', 16),
(18, 'Bruce Springsteen', 8),
(19, 'Miles Davis', 5),
(20, 'Eminem', 20);

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [51]:
%%sql
-- 5 sample audiences (this gets populated by the trigger too)
INSERT INTO Audience (audience_id, type, genre) VALUES
(1, 'Teens', 'Fiction'),
(2, 'Adults', 'Rock'),
(3, 'Adults', 'Romance'),
(4, 'Teens', 'Mystery');

Running query in 'sqlite:///library.db'

4 rows affected.

++
||
++
++

In [38]:
%%sql
INSERT INTO Events (event_id, library_name, address, room_number, description, event_date, start_time, end_time) VALUES
-- Fiction events (1-5)
(1, 'Burnaby Public Library', '6100 Willingdon Ave', 101, 'Classic Book Club: The Great Gatsby Discussion', '2023-11-15', '18:00', '20:00'),
(2, 'Burnaby Public Library', '6100 Willingdon Ave', 102, 'Fantasy Writers Workshop', '2023-11-20', '16:00', '18:00'),
(3, 'Burnaby Public Library', '6100 Willingdon Ave', 103, 'Dystopian Fiction Panel', '2023-11-25', '19:00', '21:00'),
(4, 'Burnaby Public Library', '6100 Willingdon Ave', 101, 'Teen Fiction Reading Hour', '2023-12-01', '15:00', '16:30'),
(5, 'Burnaby Public Library', '6100 Willingdon Ave', 104, 'Author Talk: Modern Fiction Trends', '2023-12-05', '18:30', '20:00'),
-- Rock events (6-10)
(6, 'Burnaby Public Library', '6100 Willingdon Ave', 201, 'Classic Rock Listening Party', '2023-11-16', '19:00', '21:00'),
(7, 'Burnaby Public Library', '6100 Willingdon Ave', 202, 'Rock Music History Lecture', '2023-11-22', '18:00', '20:00'),
(8, 'Burnaby Public Library', '6100 Willingdon Ave', 201, 'Vinyl Collectors Meetup', '2023-11-28', '17:00', '19:00'),
(9, 'Burnaby Public Library', '6100 Willingdon Ave', 203, 'Guitar Basics Workshop', '2023-12-03', '14:00', '16:00'),
(10, 'Burnaby Public Library', '6100 Willingdon Ave', 201, 'Rock Album Listening Club: The Dark Side of the Moon', '2023-12-07', '18:00', '20:00'),
-- Mystery events (11-13)
(11, 'Burnaby Public Library', '6100 Willingdon Ave', 105, 'Mystery Book Club', '2023-11-18', '18:30', '20:00'),
(12, 'Burnaby Public Library', '6100 Willingdon Ave', 105, 'True Crime Discussion Group', '2023-11-24', '19:00', '21:00'),
(13, 'Burnaby Public Library', '6100 Willingdon Ave', 106, 'How to Write a Mystery Workshop', '2023-12-02', '17:00', '19:00'),
-- Romance events (14-15)
(14, 'Burnaby Public Library', '6100 Willingdon Ave', 107, 'Romance Novel Book Club', '2023-11-21', '18:00', '20:00'),
(15, 'Burnaby Public Library', '6100 Willingdon Ave', 107, 'Historical Romance Discussion', '2023-12-04', '19:00', '21:00');

Running query in 'sqlite:///library.db'

15 rows affected.

++
||
++
++

In [53]:
%%sql
INSERT INTO RecommendedFor (event_id, audience_id) VALUES
-- Fiction events recommended to Teenager/Fiction
(1, 1), (2, 1), (3, 1), (4, 1), (5, 1),
-- Rock events recommended to Adult/Rock
(6, 2), (7, 2), (8, 2), (9, 2), (10, 2),
-- Mystery events recommended to Teenager/Mystery
(11, 4), (12, 4), (13, 4),
-- Romance events recommended to Adult/Romance
(14, 3), (15, 3);

Running query in 'sqlite:///library.db'

RuntimeError: (sqlite3.IntegrityError) UNIQUE constraint failed: RecommendedFor.event_id, RecommendedFor.audience_id
[SQL: INSERT INTO RecommendedFor (event_id, audience_id) VALUES

(1, 1), (2, 1), (3, 1), (4, 1), (5, 1),

(6, 2), (7, 2), (8, 2), (9, 2), (10, 2),

(11, 4), (12, 4), (13, 4),

(14, 3), (15, 3);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [52]:
%%sql
INSERT INTO Audience (audience_id, type, genre) VALUES
(5, 'Adults', 'Fiction')

Running query in 'sqlite:///library.db'

1 rows affected.

++
||
++
++

In [54]:
%%sql
-- This and the above insert is to also add Adult and Fiction as an audience, to make sure fiction events go to both teens and adults.
INSERT INTO RecommendedFor (event_id, audience_id) VALUES
(1, 5), (2, 5), (3, 5), (4, 5), (5, 5)

Running query in 'sqlite:///library.db'

RuntimeError: (sqlite3.IntegrityError) UNIQUE constraint failed: RecommendedFor.event_id, RecommendedFor.audience_id
[SQL: INSERT INTO RecommendedFor (event_id, audience_id) VALUES
(1, 5), (2, 5), (3, 5), (4, 5), (5, 5)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [56]:
%%sql 
SELECT * FROM RecommendedFor;

Running query in 'sqlite:///library.db'

event_id,audience_id
1,1
2,1
3,1
4,1
5,1
6,2
7,2
8,2
9,2
10,2


In [58]:
%%sql 
    SELECT * FROM Audience;

Running query in 'sqlite:///library.db'

audience_id,type,genre
1,Teens,Fiction
2,Adults,Rock
3,Adults,Romance
4,Teens,Mystery
5,Adults,Fiction
